In [1]:
from geneformer import EmbExtractor
import pickle
import pandas as pd
import anndata as ad
import numpy as np
import geneformer
with open(geneformer.tokenizer.TOKEN_DICTIONARY_FILE, 'rb') as fp:
    a = pickle.load(fp)
print(len(a))
from datasets import load_from_disk
from collections import Counter
import pickle
import scanpy as sc
from geneformer.in_silico_perturber import downsample_and_sort, \
                                 gen_attention_mask, \
                                 get_model_input_size, \
                                 load_and_filter, \
                                 load_model, \
                                 pad_tensor_list, \
                                 quant_layers, \
                                 pad_tensor

23698


In [2]:
import scanpy as sc

In [3]:
import torch

## 对simulate数据进行tokenize

In [2]:
simulation_number = '_lung2'
#pseudo_adata = sc.read_h5ad('/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset' + str(simulation_number) + '/embedding_diffusion/singlecell/pseudo/pseudo_adata.h5ad')
ST_adata = sc.read_h5ad('/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset' + str(simulation_number) + '/embedding_diffusion/ST/ST_adata.h5ad')
sc_adata = sc.read_h5ad('/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset' + str(simulation_number) + '/embedding_diffusion/sc/sc_adata.h5ad')
ST_adata.obs = ST_adata.obs.rename(columns={'Proliferating_NK/T':'Proliferating_NK_T'})

ST_adata.var['ensembl_id'] = list(ST_adata.var_names)
sc_adata.var['ensembl_id'] = list(sc_adata.var_names)
ST_adata.obs['n_counts'] = np.sum(ST_adata.X, axis=1).astype(int)
sc_adata.obs['n_counts'] = np.sum(sc_adata.X, axis=1).astype(int)

#保存成loom文件
#pseudo_adata.write_loom('/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset'+str(simulation_number)+'/embedding_diffusion/singlecell/pseudo/pseudo_adata.loom', write_obsm_varm=True)
ST_adata.write_loom('/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset'+str(simulation_number)+'/embedding_diffusion/ST/ST_adata.loom', write_obsm_varm=True)
sc_adata.write_loom('/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset'+str(simulation_number)+'/embedding_diffusion/sc/sc_adata.loom', write_obsm_varm=True)

In [5]:
simulation_number = 12
#pseudo_adata = sc.read_h5ad('/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset' + str(simulation_number) + '/embedding_diffusion/singlecell/pseudo/pseudo_adata.h5ad')
ST_adata = sc.read_h5ad('/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset' + str(simulation_number) + '/embedding_diffusion/ST/ST_adata.h5ad')
sc_adata = sc.read_h5ad('/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset' + str(simulation_number) + '/embedding_diffusion/sc/sc_adata.h5ad')

#数据预处理
name_id = pd.read_csv('/mnt/nfs/wbzhang/data/celltype_annotation/gene_info_table.csv')
name_id = name_id[(name_id.gene_type == 'protein_coding') | (name_id.gene_type == 'miRNA')]

common_genes = list(set(name_id['gene_name']).intersection(set(ST_adata.var_names)))
print(len(common_genes))
#pseudo_adata = pseudo_adata[:, common_genes]
ST_adata = ST_adata[:, common_genes]
sc_adata = sc_adata[:, common_genes]

# mapping the enseml_id
ensembl_id = []
for gene in ST_adata.var_names:
    id = name_id[name_id.gene_name == gene].ensembl_id.values[0]
    ensembl_id.append(id)
#pseudo_adata.var['ensembl_id'] = ensembl_id
ST_adata.var['ensembl_id'] = ensembl_id
sc_adata.var['ensembl_id'] = ensembl_id

# n_counts 一定要算，cell_type和organ可以没有
#pseudo_adata.obs['n_counts'] = np.sum(pseudo_adata.X, axis=1).astype(int)
ST_adata.obs['n_counts'] = np.sum(ST_adata.X, axis=1).astype(int)
sc_adata.obs['n_counts'] = np.sum(sc_adata.X, axis=1).astype(int)

#保存成loom文件
#pseudo_adata.write_loom('/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset'+str(simulation_number)+'/embedding_diffusion/singlecell/pseudo/pseudo_adata.loom', write_obsm_varm=True)
ST_adata.write_loom('/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset'+str(simulation_number)+'/embedding_diffusion/ST/ST_adata.loom', write_obsm_varm=True)
sc_adata.write_loom('/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset'+str(simulation_number)+'/embedding_diffusion/sc/sc_adata.loom', write_obsm_varm=True)

795


/tmp/ipykernel_54767/800111254.py:22: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  ST_adata.var['ensembl_id'] = ensembl_id
/tmp/ipykernel_54767/800111254.py:23: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  sc_adata.var['ensembl_id'] = ensembl_id


In [3]:
# 开始tokenize# 开始tokenize
from geneformer import TranscriptomeTokenizer
data_path = '/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset'+str(simulation_number)+'/embedding_diffusion/'

for t in ['sc', 'ST']:
    loom_data_directory = data_path + t + '/'
    output_directory = loom_data_directory
    output_prefix = 'tokenized_adata'
    tk = TranscriptomeTokenizer(nproc=16)
    tk.tokenize_data(loom_data_directory, output_directory, output_prefix)

Tokenizing /mnt/nfs/wbzhang/stdgcn/data/simulate/dataset_lung2/embedding_diffusion/sc/sc_adata.loom
/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset_lung2/embedding_diffusion/sc/sc_adata.loom has no column attribute 'filter_pass'; tokenizing all cells.


Map (num_proc=16):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Tokenizing /mnt/nfs/wbzhang/stdgcn/data/simulate/dataset_lung2/embedding_diffusion/ST/ST_adata.loom
/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset_lung2/embedding_diffusion/ST/ST_adata.loom has no column attribute 'filter_pass'; tokenizing all cells.


Map (num_proc=16):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

## Raw code for geneformer embedding

In [4]:
from datasets import Dataset
def WB_padding(input_data, length_final, pad_token_id):
    data = input_data['input_ids']
    new_input_ids = []
    lengths = []
    for cell in data:
        if len(cell) < length_final:
            cell_tensor = torch.tensor(cell)
            cell_tensor = pad_tensor(cell_tensor, pad_token_id, length_final)
            new_input_ids.append(list(cell_tensor.numpy()))
        else:
            new_input_ids.append(cell[:length_final])
        lengths.append(length_final)
    dataset_dict = {"input_ids": new_input_ids, 'length': lengths}
    output_dataset = Dataset.from_dict(dataset_dict)

    return output_dataset
#WB_padding(filtered_input_data, length_final=1000, pad_token_id=0)

In [7]:
simulation_number = '_lung2'
target = 'sc'

from geneformer.tokenizer import TOKEN_DICTIONARY_FILE
from tqdm.notebook import trange
import torch
import os
model_type="diffusion_12L"
num_classes=0
emb_mode="cell"
cell_emb_style = None #利用 mean 的方式，后面需要改进
filter_data=None
max_ncells=30000
emb_layer=-1
emb_label=None
labels_to_plot=None
forward_batch_size=100
nproc=128
token_dictionary_file=TOKEN_DICTIONARY_FILE

with open(token_dictionary_file, "rb") as f:
    gene_token_dict = pickle.load(f)
pad_token_id = gene_token_dict.get("<pad>")

pretrain_model_path = '/mnt/nfs/xz/diffusion/db/Diffusion-BERT/model_out/230901_025119_model_name_gene_lr_5e-06_seed_42_numsteps_128_sample_Categorical_schedule_mutual_hybridlambda_0.01_wordfreqlambda_0.3_fromscratch_False_timestep_none_ckpts/best_0_14999.th'
#pretrain_model_path = '/mnt/nfs/xz/diffusion/db/Diffusion-BERT/model_out/230909_225342_model_name_gene_lr_1e-08_seed_42_numsteps_2048_sample_Categorical_schedule_mutual_hybridlambda_0.01_wordfreqlambda_0.3_fromscratch_False_timestep_none_ckpts/best_0_237404.th'
data_path = '/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset'+str(simulation_number)+'/embedding_diffusion/'+target+'/tokenized_adata.dataset'
output_directory = '/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset'+str(simulation_number)+'/embedding_diffusion/embed_output'
os.makedirs(output_directory, exist_ok=True)

# extrac_embs
model_directory = pretrain_model_path
input_data_file = data_path
output_directory = output_directory
output_prefix = "emb_results"

filtered_input_data = load_and_filter(filter_data, nproc, input_data_file)
downsampled_data = downsample_and_sort(filtered_input_data, max_ncells) #取出max cell 的数据

# 修改： 对downsample_data数据进行 2048 padding处理
WB_input_data = WB_padding(filtered_input_data, length_final=2048, pad_token_id=pad_token_id)

model = load_model(model_type, num_classes, model_directory)
layer_to_quant = quant_layers(model)+ emb_layer 
print('layer:', layer_to_quant) # embedding 的层数

#get_embs
model_input_size = get_model_input_size(model)
print('model_input_size:', model_input_size)
total_batch_length = len(filtered_input_data)
if ((total_batch_length-1)/forward_batch_size).is_integer():
    forward_batch_size = forward_batch_size-1
print('forward_batch_size:', forward_batch_size)
#max_len = max(filtered_input_data['length'])
#start embedding
#embs_total = torch.zeros([len(WB_input_data), 512*128]).to('cuda')
embs_total = torch.zeros([len(WB_input_data), 2048*512]).to('cuda')
for i in trange(0, total_batch_length, forward_batch_size):
    max_range = min(i+forward_batch_size, total_batch_length)
    # 数据修改在这里
    minibatch = WB_input_data.select([i for i in range(i, max_range)])
    max_len = max(minibatch["length"]) # 取最长的句子长度，作为后续padding的标准
    original_lens = torch.tensor(minibatch["length"]).to("cuda")
    minibatch.set_format(type="torch")
    input_data_minibatch = minibatch["input_ids"]
    input_data_minibatch = pad_tensor_list(input_data_minibatch, 
                                       max_len, 
                                       pad_token_id, 
                                       model_input_size)
    with torch.no_grad():
        outputs = model(
            input_ids = input_data_minibatch.to("cuda"),
            attention_mask = gen_attention_mask(minibatch)
        )
    embs_i = outputs.hidden_states[layer_to_quant]
    if cell_emb_style == 'maxpooling':
        maxpool = torch.nn.MaxPool2d(4, stride=4, padding=1)
        embed_pooling = maxpool(embs_i)
        embs_total[i:(i+forward_batch_size)] = embed_pooling.flatten(1)
    elif cell_emb_style == 'avgpooling':
        avgpool = torch.nn.AvgPool2d(4, stride=4, padding=1)
        embed_pooling = avgpool(embs_i)
        embs_total[i:(i+forward_batch_size)] = embed_pooling.flatten(1)
    else:
        embed_pooling = None
        embs_total[i:(i+forward_batch_size)] = embs_i.flatten(1)
    del outputs
    del minibatch
    del input_data_minibatch
    del embs_i
    del embed_pooling
    torch.cuda.empty_cache()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /mnt/nfs/xz/geneformer/Geneformer34m/examples/pretraining_new_model/output_own/models/230813_170132_geneformer_30M_L12_emb512_SL2048_E3_B12_LR0.0005_LSlinear_WU10000_Oadamw_DS64/models and are newly initialized: ['bert.pooler.dense.bias', 'classifier.weight', 'bert.pooler.dense.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


layer: 11
model_input_size: 2048
forward_batch_size: 100


  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
ST_embed = embs_total.cpu()
print(ST_embed.shape)
del embs_total
torch.cuda.empty_cache()

torch.Size([1000, 1048576])


In [8]:
sc_embed = embs_total.cpu()
print(sc_embed.shape)
del embs_total
torch.cuda.empty_cache()

torch.Size([10000, 1048576])


In [9]:
print(ST_embed.shape)
print(sc_embed.shape)

torch.Size([1000, 1048576])
torch.Size([10000, 1048576])


In [10]:
# 一起PCA
from sklearn.decomposition import PCA

X = torch.cat((ST_embed, sc_embed), dim=0).numpy()
pca=PCA(n_components=1024)
pca_results=pca.fit_transform(X)

In [11]:
pca_results.shape

(11000, 1024)

In [12]:
import torch
embeds = torch.tensor(pca_results, dtype=torch.float64)
torch.save(embeds, '/mnt/nfs/wbzhang/stdgcn/data/simulate/dataset'+str(simulation_number)+'/embedding_diffusion/12L_scST_total_pca1024_embed.pth')